### Imports and Setup

In [34]:
from herbie import Herbie, FastHerbie, wgrib2
import pandas as pd
import xarray as xr
import geojson

### Helper Functions

In [84]:
# Takes a dict of vars you want to extract, and the corresponding granularities
# Uses XARRAY accessors to limit them
def download_parameters(parameters: dict, fh: FastHerbie) -> None:
    fields = [f":{param}:{level}" for param, level in parameters.items()]
    param_regex = f"^(?:{'|'.join(fields)})"
    print(param_regex)
    fh.download(param_regex)

# What it does
# Inputs/Parameters names and descriptions
# Returns
    

In [48]:
# Takes a JSON Object
def limit_geographic_range(geojson_path: str) -> None:
    with open(geojson_path) as f:
        poly = geojson.load(f)
        bounds = list(geojson.utils.coords(poly))
        long, lat = zip(*bounds)
        return (min(long), max(long), min(lat), max(lat))
            

### Parameter Selection

In [87]:
model = 'hrrr' # Use HRRR Model
product = 'sfc' # Use 2D surface level fields; 3-km resolution
date_range = pd.date_range(
    start="2020-02-01 00:00",
    end="2020-02-02 00:00",
    freq="1h"
)

# Parameter Names scoped from here: 
# https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfsfcf00.grib2.shtml
parameters = {
    'TMP' : 'surface', # Temperature - Hourly
    'RH' : '2 m above ground', # Relative Humidity - Hourly
    'WIND': '10 m above ground', # Wind Speed - 0-0 day max f 
    'APCP': 'surface', # Total Precip - 0-0 day acc f 
    'DSWRF': 'surface', # Downward Short-Wave Radiation Flux - Hourly
    'DLWRF': 'surface' # Downward Long-Wave Rad. Flux - Hourly
}

fh = FastHerbie(date_range, model=model, product=product, fxx=[1,2])
download_parameters(parameters, fh)
fh.objects

^(?::TMP:surface|:RH:2 m above ground|:WIND:10 m above ground|:APCP:surface|:DSWRF:surface|:DLWRF:surface)


[▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 00:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 00:00 UTC F02 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 01:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 01:00 UTC F02 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 02:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 02:00 UTC F02 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 03:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 03:00 UTC F02 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 04:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 04:00 UTC F02 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 05:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2020-Feb-01 05:00 UT

In [4]:
h = Herbie('2023-01-01')

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [25]:
h.download("^(?::TMP:surface|:RH:2 m above ground|:WIND:10 m above ground|:APCP:surface|:DSWRF:surface|:DLWRF:surface)")

PosixPath('/Users/jesse/data/hrrr/20230101/subset_18ef59c7__hrrr.t00z.wrfsfcf00.grib2')

In [26]:
h.inventory("^(?::TMP:surface|:RH:2 m above ground|:WIND:10 m above ground|:APCP:surface|:DSWRF:surface|:DLWRF:surface)")

,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
63,64,34940256,36222332.0,34940256-36222332,2023-01-01,2023-01-01,TMP,surface,anl,:TMP:surface:anl
74,75,44234631,45804243.0,44234631-45804243,2023-01-01,2023-01-01,RH,2 m above ground,anl,:RH:2 m above ground:anl
78,79,50884588,52092416.0,50884588-52092416,2023-01-01,2023-01-01,WIND,10 m above ground,0-0 day max fcst,:WIND:10 m above ground:0-0 day max fcst
83,84,54442559,54442770.0,54442559-54442770,2023-01-01,2023-01-01,APCP,surface,0-0 day acc fcst,:APCP:surface:0-0 day acc fcst
122,123,85698959,86360826.0,85698959-86360826,2023-01-01,2023-01-01,DSWRF,surface,anl,:DSWRF:surface:anl
123,124,86360827,88445279.0,86360827-88445279,2023-01-01,2023-01-01,DLWRF,surface,anl,:DLWRF:surface:anl


In [28]:
myFile = h.get_localFilePath("^(?::TMP:surface|:RH:2 m above ground|:WIND:10 m above ground|:APCP:surface|:DSWRF:surface|:DLWRF:surface)")

In [29]:
myFile

PosixPath('/Users/jesse/data/hrrr/20230101/subset_18ef59c7__hrrr.t00z.wrfsfcf00.grib2')

In [31]:
print(wgrib2.inventory(myFile))

1:0:d=2023010100:TMP:surface:anl:
2:1282077:d=2023010100:RH:2 m above ground:anl:
3:2851690:d=2023010100:WIND:10 m above ground:0-0 day max fcst:
4:4059519:d=2023010100:APCP:surface:0-0 day acc fcst:
5:4059731:d=2023010100:DSWRF:surface:anl:
6:4721599:d=2023010100:DLWRF:surface:anl:



In [32]:
wgrib2.vector_relative(myFile)

All winds are grid-relative winds.


{'winds(grid)'}

In [53]:
skagit_bounds = limit_geographic_range("skagit_boundaries.json")
subset_file = wgrib2.region(myFile, skagit_bounds, name="skagit-basin")
subset_file

PosixPath('/Users/jesse/data/hrrr/20230101/skagit-basin_subset_18ef59c7__hrrr.t00z.wrfsfcf00.grib2')

In [71]:
skagit_bounds

(-123.409663, -118.092281, 47.543471, 50.06801)

In [75]:
print(wgrib2.inventory(subset_file))

1:0:d=2023010100:TMP:surface:anl:
2:16992:d=2023010100:RH:2 m above ground:anl:
3:38682:d=2023010100:WIND:10 m above ground:0-0 day max fcst:
4:55995:d=2023010100:APCP:surface:0-0 day acc fcst:
5:56238:d=2023010100:DSWRF:surface:anl:
6:77491:d=2023010100:DLWRF:surface:anl:

